In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn

# Definir el modelo de regresión
class BertRegressionModel(nn.Module):
    def __init__(self, bert_model):
        super(BertRegressionModel, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(p=0.3)
        self.regressor = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        output = self.regressor(pooled_output)
        return output.squeeze(-1)

# Cargar el modelo y el tokenizer
@st.cache(allow_output_mutation=True)
def load_model():
    tokenizer = AutoTokenizer.from_pretrained('modelo_beto_regresion')
    bert_model = AutoModel.from_pretrained('modelo_beto_regresion')
    model = BertRegressionModel(bert_model)
    model.load_state_dict(torch.load('modelo_beto_regresion/bert_regression_model.pt', map_location=torch.device('cpu')))
    model.eval()
    return tokenizer, model

tokenizer, model = load_model()